In [ ]:
# Install all required dependencies
!pip install transformers torch accelerate sentencepiece -q
!pip install datasets -q

In [ ]:
# Import all necessary libraries
import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    GPT2LMHeadModel,
    GPT2Tokenizer
)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class SentimentAnalyzer:
    """
    Sentiment Analysis using pre-trained RoBERTa model
    Classifies text into: Positive, Negative, or Neutral
    """

    def __init__(self):
        print("Loading sentiment analysis model...")
        # Using cardiffnlp/twitter-roberta-base-sentiment-latest for robust sentiment detection
        self.model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
        self.sentiment_pipeline = pipeline(
            "sentiment-analysis",
            model=self.model_name,
            tokenizer=self.model_name
        )
        print("✓ Sentiment analyzer loaded successfully!")

    def analyze(self, text):
        """
        Analyze sentiment of input text
        Returns: dict with label and score
        """
        result = self.sentiment_pipeline(text)[0]

        # Map labels to readable format
        label_mapping = {
            'LABEL_0': 'negative',
            'LABEL_1': 'neutral',
            'LABEL_2': 'positive',
            'negative': 'negative',
            'neutral': 'neutral',
            'positive': 'positive'
        }

        sentiment = label_mapping.get(result['label'].lower(), result['label'])
        score = result['score']

        return {
            'sentiment': sentiment,
            'confidence': score
        }

In [ ]:
class SentimentTextGenerator:
    """
    Text Generation Model with Sentiment Control
    Uses GPT-2 with sentiment-specific prompts
    """

    def __init__(self):
        print("Loading text generation model...")
        # Using DistilGPT2 for faster, efficient text generation
        self.model_name = "distilgpt2"
        self.tokenizer = GPT2Tokenizer.from_pretrained(self.model_name)
        self.model = GPT2LMHeadModel.from_pretrained(self.model_name)

        # Set padding token
        self.tokenizer.pad_token = self.tokenizer.eos_token

        # Define sentiment-specific prompts to guide generation
        self.sentiment_prompts = {
            'positive': [
                "This is wonderful because",
                "I am delighted to share that",
                "The amazing thing is",
                "Fortunately,",
                "What a fantastic"
            ],
            'negative': [
                "Unfortunately,",
                "This is disappointing because",
                "Sadly,",
                "The problem is that",
                "It's frustrating that"
            ],
            'neutral': [
                "It should be noted that",
                "The fact is that",
                "Generally speaking,",
                "According to reports,",
                "In this case,"
            ]
        }

        print("✓ Text generator loaded successfully!")

    def generate(self, prompt, sentiment, max_length=150, temperature=0.8, num_return=1):
        """
        Generate text aligned with specified sentiment

        Args:
            prompt: Input prompt text
            sentiment: Target sentiment ('positive', 'negative', 'neutral')
            max_length: Maximum length of generated text
            temperature: Controls randomness (0.7-1.0 recommended)
            num_return: Number of sequences to generate

        Returns:
            Generated text aligned with sentiment
        """
        # Select sentiment-specific prefix
        import random
        sentiment_prefix = random.choice(self.sentiment_prompts.get(sentiment, ['']))

        # Combine prompt with sentiment guidance
        full_prompt = f"{sentiment_prefix} {prompt}"

        # Encode input
        inputs = self.tokenizer.encode(full_prompt, return_tensors='pt')

        # Set attention mask
        attention_mask = torch.ones(inputs.shape, dtype=torch.long)

        # Generate text
        outputs = self.model.generate(
            inputs,
            attention_mask=attention_mask,
            max_length=max_length,
            min_length=50,
            temperature=temperature,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            num_return_sequences=num_return,
            pad_token_id=self.tokenizer.eos_token_id,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )

        # Decode generated text
        generated_texts = []
        for output in outputs:
            text = self.tokenizer.decode(output, skip_special_tokens=True)
            generated_texts.append(text)

        return generated_texts[0] if num_return == 1 else generated_texts

In [ ]:
class AITextGenerator:
    """
    Complete AI Text Generator System
    Combines sentiment analysis and sentiment-aligned text generation
    """

    def __init__(self):
        print("\n" + "="*50)
        print("INITIALIZING AI TEXT GENERATOR SYSTEM")
        print("="*50 + "\n")

        self.sentiment_analyzer = SentimentAnalyzer()
        self.text_generator = SentimentTextGenerator()

        print("\n✓ System ready!")
        print("="*50 + "\n")

    def generate_from_prompt(self, prompt, max_length=150, temperature=0.8,
                            manual_sentiment=None, num_outputs=1):
        """
        Main function to generate text based on prompt sentiment

        Args:
            prompt: User input prompt
            max_length: Maximum length of generated text
            temperature: Controls creativity (0.7-1.0)
            manual_sentiment: Override detected sentiment ('positive', 'negative', 'neutral')
            num_outputs: Number of text variations to generate

        Returns:
            dict with sentiment info and generated text
        """
        print(f"\nInput Prompt: '{prompt}'")
        print("-" * 50)

        # Step 1: Analyze sentiment (if not manually specified)
        if manual_sentiment:
            sentiment = manual_sentiment
            confidence = 1.0
            print(f"Manual Sentiment: {sentiment.upper()}")
        else:
            sentiment_result = self.sentiment_analyzer.analyze(prompt)
            sentiment = sentiment_result['sentiment']
            confidence = sentiment_result['confidence']
            print(f"Detected Sentiment: {sentiment.upper()} (confidence: {confidence:.2%})")

        print(f"Generating {num_outputs} text(s) aligned with {sentiment.upper()} sentiment...")
        print("-" * 50)

        # Step 2: Generate sentiment-aligned text
        generated_texts = self.text_generator.generate(
            prompt=prompt,
            sentiment=sentiment,
            max_length=max_length,
            temperature=temperature,
            num_return=num_outputs
        )

        # Format output
        if num_outputs == 1:
            print(f"\nGenerated Text:\n{generated_texts}\n")
        else:
            for i, text in enumerate(generated_texts, 1):
                print(f"\nVariation {i}:\n{text}\n")

        return {
            'prompt': prompt,
            'detected_sentiment': sentiment,
            'confidence': confidence,
            'generated_text': generated_texts
        }

    def generate_with_all_sentiments(self, prompt, max_length=150):
        """
        Generate text for all three sentiments to compare
        """
        print("\n" + "="*50)
        print("GENERATING TEXT FOR ALL SENTIMENTS")
        print("="*50)

        results = {}
        for sentiment in ['positive', 'negative', 'neutral']:
            print(f"\n{'='*20} {sentiment.upper()} {'='*20}")
            result = self.generate_from_prompt(
                prompt=prompt,
                max_length=max_length,
                manual_sentiment=sentiment
            )
            results[sentiment] = result

        return results

In [ ]:
# Initialize the complete AI Text Generator system
ai_generator = AITextGenerator()


INITIALIZING AI TEXT GENERATOR SYSTEM

Loading sentiment analysis model...


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✓ Text generator loaded successfully!

✓ System ready!



In [ ]:
print("\n" + "="*50)
print("EXAMPLE 1: AUTO SENTIMENT DETECTION")
print("="*50)

result1 = ai_generator.generate_from_prompt(
    prompt="The weather today is absolutely beautiful",
    max_length=120,
    temperature=0.8
)


EXAMPLE 1: AUTO SENTIMENT DETECTION

Input Prompt: 'The weather today is absolutely beautiful'
--------------------------------------------------
Detected Sentiment: POSITIVE (confidence: 98.22%)
Generating 1 text(s) aligned with POSITIVE sentiment...
--------------------------------------------------

Generated Text:
This is wonderful because The weather today is absolutely beautiful. But with all the humidity at home and in our winter, when we're sleeping on a bedside there's no light to illuminate that night or even for whatever reason."
It was also like this: It wasn't just an early morning rainstorm (the Weather Channel reports) but it had its own unique moment of "great relief" as one day passed by around 5 p—almost five days after Hurricane Irma hit Florida earlier tonight — before hitting East Coast last week! Just imagine what would have happened if you'd been caught watching these incredible



In [ ]:
print("\n" + "="*50)
print("EXAMPLE 2: MANUAL SENTIMENT (NEGATIVE)")
print("="*50)

result2 = ai_generator.generate_from_prompt(
    prompt="The meeting was cancelled",
    max_length=120,
    temperature=0.8,
    manual_sentiment='negative'
)


EXAMPLE 2: MANUAL SENTIMENT (NEGATIVE)

Input Prompt: 'The meeting was cancelled'
--------------------------------------------------
Manual Sentiment: NEGATIVE
Generating 1 text(s) aligned with NEGATIVE sentiment...
--------------------------------------------------

Generated Text:
The problem is that The meeting was cancelled due to poor attendance, and the other two members of Parliament were unable to attend their own meetings (which have no bearing on who will be elected). I think it’s important for a representative parliament in this country if there are any representatives from outside Europe. As an example: while many people voted against being allowed to hold more than five events annually with similar frequency they would vote at home when these groups could not bring out themselves as democratic nations – which means voting abroad does not matter much because everyone should go away without having your name taken care either way



In [ ]:
print("\n" + "="*50)
print("EXAMPLE 3: MULTIPLE VARIATIONS")
print("="*50)

result3 = ai_generator.generate_from_prompt(
    prompt="Technology is changing our world",
    max_length=130,
    temperature=0.9,
    num_outputs=2
)


EXAMPLE 3: MULTIPLE VARIATIONS

Input Prompt: 'Technology is changing our world'
--------------------------------------------------
Detected Sentiment: POSITIVE (confidence: 85.02%)
Generating 2 text(s) aligned with POSITIVE sentiment...
--------------------------------------------------

Variation 1:
The amazing thing is Technology is changing our world. With the advances in all of technology, it’s possible to make a living with this!
And as if we don't know what you want or need more information about why: http://en.wikipediausercontent/wiki//Tech.org
If there's something wrong I'll just look into that and try to help fix them (in fact they should be).


Variation 2:
The amazing thing is Technology is changing our world, even if it doesn't affect the people we love."
In an interview with TIME Magazine in August 2010, he explained that Google was "a very good example of what a truly smart company can do," and pointed out how great companies make themselves. The tech firm's technology

In [ ]:
print("\n" + "="*50)
print("EXAMPLE 4: ALL SENTIMENTS COMPARISON")
print("="*50)

all_sentiments = ai_generator.generate_with_all_sentiments(
    prompt="Working from home",
    max_length=120
)


EXAMPLE 4: ALL SENTIMENTS COMPARISON

GENERATING TEXT FOR ALL SENTIMENTS

==================== POSITIVE ====================

Input Prompt: 'Working from home'
--------------------------------------------------
Manual Sentiment: POSITIVE
Generating 1 text(s) aligned with POSITIVE sentiment...
--------------------------------------------------

Generated Text:
Fortunately, Working from home is a great way to learn and communicate.
The new study also confirms that the brain has significant benefits for learning in different aspects of everyday life: "Working out with friends will make you more able not to worry about problems like traffic or health."


==================== NEGATIVE ====================

Input Prompt: 'Working from home'
--------------------------------------------------
Manual Sentiment: NEGATIVE
Generating 1 text(s) aligned with NEGATIVE sentiment...
--------------------------------------------------

Generated Text:
It's frustrating that Working from home, working wit

In [ ]:
def generate_sentiment_text(prompt, sentiment_option='auto',
                           text_length=150, creativity=0.8):
    """
    Simplified function for Streamlit frontend integration

    Args:
        prompt: User input text
        sentiment_option: 'auto', 'positive', 'negative', or 'neutral'
        text_length: Desired length (50-300)
        creativity: Temperature for generation (0.5-1.5)

    Returns:
        tuple: (sentiment, confidence, generated_text)
    """
    manual = None if sentiment_option == 'auto' else sentiment_option

    result = ai_generator.generate_from_prompt(
        prompt=prompt,
        max_length=text_length,
        temperature=creativity,
        manual_sentiment=manual
    )

    return (
        result['detected_sentiment'],
        result['confidence'],
        result['generated_text']
    )

In [ ]:
print("\n" + "="*50)
print("TESTING INTERACTIVE FUNCTION")
print("="*50)

sentiment, conf, text = generate_sentiment_text(
    prompt="Artificial intelligence in healthcare",
    sentiment_option='positive',
    text_length=140,
    creativity=0.85
)

print(f"\nSentiment: {sentiment}")
print(f"Confidence: {conf:.2%}")
print(f"Generated: {text}")


TESTING INTERACTIVE FUNCTION

Input Prompt: 'Artificial intelligence in healthcare'
--------------------------------------------------
Manual Sentiment: POSITIVE
Generating 1 text(s) aligned with POSITIVE sentiment...
--------------------------------------------------

Generated Text:
Fortunately, Artificial intelligence in healthcare will continue to grow.
“The world has seen a huge increase in medical research funding and we are seeing an exponential growth of the supply chain because it is now more reliable than ever before for most people," said Dr David Nixon from Oxford University Hospitals Research Unit (Oxford).
However, despite its large population size, artificial intelligence still remains just as expensive globally compared with other forms of technology such that human brain can use AI or even computers – all while making billions on investment worldwide."With machines becoming increasingly ubiquitous through their application applications they need not only be smarter bu

In [ ]:
def save_models(save_path='./ai_text_generator_models'):
    """
    Save models for deployment
    """
    import os
    os.makedirs(save_path, exist_ok=True)

    print(f"\nSaving models to {save_path}...")

    # Save text generator
    ai_generator.text_generator.model.save_pretrained(f"{save_path}/text_generator")
    ai_generator.text_generator.tokenizer.save_pretrained(f"{save_path}/text_generator")

    print("✓ Models saved successfully!")

# Uncomment the line below to save models
save_models()


Saving models to ./ai_text_generator_models...
✓ Models saved successfully!


Custom Text

In [ ]:
# Test with your own prompt here
print("\n" + "="*50)
print("CUSTOM TEST")
print("="*50)

# Modify these parameters as needed
custom_prompt = "Machine learning is revolutionizing industries"
custom_sentiment = 'auto'  # Options: 'auto', 'positive', 'negative', 'neutral'
custom_length = 150
custom_creativity = 0.8

custom_result = ai_generator.generate_from_prompt(
    prompt=custom_prompt,
    max_length=custom_length,
    temperature=custom_creativity,
    manual_sentiment=None if custom_sentiment == 'auto' else custom_sentiment
)


CUSTOM TEST

Input Prompt: 'Machine learning is revolutionizing industries'
--------------------------------------------------
Detected Sentiment: POSITIVE (confidence: 92.95%)
Generating 1 text(s) aligned with POSITIVE sentiment...
--------------------------------------------------

Generated Text:
I am delighted to share that Machine learning is revolutionizing industries. I can‬t wait to see what the future holds for me, but today we must be able to use it at a much higher level without ever being overwhelmed with problems and obstacles."
The most interesting part of this post: Is there an important shift happening in automation? "We have seen large numbers of people start doing their own work based on existing solutions or techniques," writes Dr. Michael Aaronson, Phd., senior vice president for AI development/engineering research (AI). The two leading experts who are all well-known users of machine learning believe those skills will become increasingly available as they go online